<a href="https://colab.research.google.com/github/Rioto3/portfolio-kpi/blob/feature%2Fanalytics-dashboard/analysis/portfolio_analytics_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリ
!pip install pandas
!pip install plotly
from google.cloud import bigquery
import pandas as pd
import plotly.express as px


In [ ]:
# 1. 基本的なページビュー分析
page_views_query = """
SELECT
    DATE(timestamp) as date,
    COUNT(*) as pageviews,
    COUNT(DISTINCT user_pseudo_id) as unique_visitors,
    COUNT(*) / COUNT(DISTINCT user_pseudo_id) as pages_per_visitor
FROM `digital-portfolio-analytics-bq.analytics_449733397287.events_*`
WHERE event_name = 'page_view'
GROUP BY date
ORDER BY date DESC
"""

# 2. セッション分析
session_analysis_query = """
SELECT
    DATE(timestamp) as date,
    COUNT(DISTINCT session_id) as total_sessions,
    AVG(engagement_time_msec) / 1000 as avg_session_duration_sec
FROM `digital-portfolio-analytics-bq.analytics_449733397287.events_*`
GROUP BY date
ORDER BY date DESC
"""

# 3. ユーザー行動分析
user_behavior_query = """
SELECT
    DATE(timestamp) as date,
    page_location,
    COUNT(*) as page_views,
    COUNT(DISTINCT user_pseudo_id) as unique_visitors,
    AVG(engagement_time_msec) / 1000 as avg_time_on_page_sec
FROM `digital-portfolio-analytics-bq.analytics_449733397287.events_*`
WHERE event_name = 'page_view'
GROUP BY date, page_location
ORDER BY date DESC, page_views DESC
"""

# 4. 地域別アクセス分析
location_analysis_query = """
SELECT
    country,
    city,
    COUNT(DISTINCT user_pseudo_id) as unique_visitors,
    COUNT(*) as total_events
FROM `digital-portfolio-analytics-bq.analytics_449733397287.events_*`
GROUP BY country, city
ORDER BY unique_visitors DESC
"""

# データ取得用の共通関数
def fetch_data(query):
    return pd.read_gbq(query, project_id='digital-portfolio-analytics-bq')

# 可視化用の関数群
def plot_time_series(df, x_col, y_col, title):
    fig = px.line(df, x=x_col, y=y_col, title=title)
    fig.show()

def plot_bar(df, x_col, y_col, title):
    fig = px.bar(df, x=x_col, y=y_col, title=title)
    fig.show()

def create_summary_dashboard(df_page_views, df_sessions):
    # メイン指標のサマリー
    total_visitors = df_page_views['unique_visitors'].sum()
    total_pageviews = df_page_views['pageviews'].sum()
    avg_session_duration = df_sessions['avg_session_duration_sec'].mean()

    print(f"総訪問者数: {total_visitors}")
    print(f"総ページビュー: {total_pageviews}")
    print(f"平均セッション時間: {avg_session_duration:.2f}秒")

# 分析実行例
"""
# データ取得
df_page_views = fetch_data(page_views_query)
df_sessions = fetch_data(session_analysis_query)
df_behavior = fetch_data(user_behavior_query)
df_location = fetch_data(location_analysis_query)

# ダッシュボード生成
create_summary_dashboard(df_page_views, df_sessions)

# トレンド可視化
plot_time_series(df_page_views, 'date', 'unique_visitors', '日別ユニークビジター数')
plot_time_series(df_page_views, 'date', 'pageviews', '日別ページビュー数')
plot_time_series(df_sessions, 'date', 'avg_session_duration_sec', '平均セッション時間の推移')
"""